# Необходимые библиотеки

In [7]:
import pandas as pd
import numpy as np

In [8]:
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
import warnings

In [10]:
warnings.filterwarnings('ignore')

# Загрузка данных

In [11]:
data = pd.read_csv('hh ru DATASET.csv')
data.head()

,vacancy_id,employer_id,employees_number,creation_date,work_schedule,employment,length_of_employment,compensation_from,compensation_to,accept_teenagers,...,young_response_count,young_female_response_count,young_male_response_count,invitation_count,female_invitation_count,male_invitation_count,young_invitation_count,young_female_invitation_count,young_male_invitation_count,industry_id_list
0,77424192,2571258,300,2023-02-27,full_day,full,up_to_3_years,35000.0,0.0,False,...,4,4,0,21,21,0,0,0,0,[48]
1,78161908,4809555,50,2023-03-17,shift,full,no_experience,22500.0,NaN,False,...,5,4,1,26,26,0,2,2,0,[None]
2,79496395,5667513,5,2023-04-19,shift,full,up_to_3_years,24000.0,30000.0,False,...,2,2,0,3,3,0,0,0,0,[None]
3,87102246,1072323,0,2023-09-21,full_day,full,up_to_3_years,50000.0,NaN,False,...,0,0,0,9,7,2,0,0,0,"[7, 8]"
4,76367381,1160192,0,2023-01-30,full_day,full,up_to_3_years,55000.0,150000.0,False,...,0,0,0,49,39,10,0,0,0,"[5, 13]"


In [12]:
original_shape = data.shape

# Обработка выбросов

In [13]:
#num_cols = data.select_dtypes('number').columns.drop(['vacancy_id', 'employer_id'])

In [14]:
#fig, axis = plt.subplots(1, 5, figsize=(20, 5))

#sns.boxplot(data=data['employees_number'], ax=axis[0])
#sns.boxplot(data=data['compensation_from'], ax=axis[1])
#sns.boxplot(data=data['compensation_to'], ax=axis[2])
#sns.boxplot(data=data['response_count'], ax=axis[3])
#sns.boxplot(data=data['invitation_count'], ax=axis[4])

#plt.tight_layout()
#plt.show()

In [15]:
#garbage = []

#for num_col in num_cols:
#    upper_bound = data[num_col].quantile(0.975)
#    if 'compensation' in num_col:
#        lower_bound = data[data[num_col] != 0][num_col].quantile(0.003)
#        ejections = data[(data[num_col] > upper_bound) | (data[num_col] < lower_bound) & (data[num_col] != 0)]
#    else:
#        ejections = data[data[num_col] > upper_bound]
#
#    garbage.extend(ejections.index.tolist())

#garbage = list(set(garbage))
#data = data.drop(garbage).reset_index(drop=True)

In [16]:
#fig, axis = plt.subplots(1, 5, figsize=(20, 5))

#sns.boxplot(data=data['employees_number'], ax=axis[0])
#sns.boxplot(data=data['compensation_from'], ax=axis[1])
#sns.boxplot(data=data['compensation_to'], ax=axis[2])
#sns.boxplot(data=data['response_count'], ax=axis[3])
#sns.boxplot(data=data['invitation_count'], ax=axis[4])

#plt.tight_layout()
#plt.show()

In [17]:
round(1 - data.shape[0] / original_shape[0], 3)

0.0

# Удаление некорректных данных

### Нами было обнаружено несоответствие количеств откликов и/или приглашений

In [18]:
len(data[data['response_count'] != data['female_response_count'] + data['male_response_count']])

9389

In [19]:
len(data[data['young_response_count'] != data['young_female_response_count'] + data['young_male_response_count']])

66

In [20]:
len(data[data['invitation_count'] != data['female_invitation_count'] + data['male_invitation_count']])

4775

In [21]:
len(data[data['young_invitation_count'] != data['young_female_invitation_count'] + data['young_male_invitation_count']])

35

In [22]:
data = data[data['response_count'] == data['female_response_count'] + data['male_response_count']]
data = data[data['young_response_count'] == data['young_female_response_count'] + data['young_male_response_count']]
data = data[data['invitation_count'] == data['female_invitation_count'] + data['male_invitation_count']]
data = data[data['young_invitation_count'] == data['young_female_invitation_count'] + data['young_male_invitation_count']]

### Также проверим, что количество всех приглашенных больше количества молодых приглашенных

In [23]:
all(data['response_count'] >= data['young_response_count']), all(data['invitation_count'] >= data['young_invitation_count'])

(True, True)

### Данные неточности показывали некорректность данных, поэтому их мы удалили, а не заменили

# Удаление дубликатов

На дубликатах строится наш исследовательский вопрос, поэтому их мы не удаляем, глобальных же дубликатов не обнаружено

In [24]:
sum(data.duplicated())

0

# Обработка пропусков

In [25]:
data.isna().sum()[data.isnull().sum() != 0]

compensation_from     32632
compensation_to      115415
dtype: int64

Пропуски наблюдаются только в признаках, связанных с зарплатой. 

Такие пропуски объясняются нежеланием работадателя устанавливать однозначные границы зарплаты или простым игнорированием

Помимо этого так же есть много нулевых значений в compensation_from и compensation_to

In [26]:
data[data['compensation_from'] == 0].shape[0]

6504

In [27]:
data[data['compensation_to'] == 0].shape[0]

50690

Их также необходимо обратить в nan, так как 0 - это всё равно, что отсутствие данных

Мы также проверили на всякий случай, если 0 - зарплаты волонтёров, так как им не платят, но оказалось, что это не так

In [28]:
data[data['employment'] == 'volonteer']

,vacancy_id,employer_id,employees_number,creation_date,work_schedule,employment,length_of_employment,compensation_from,compensation_to,accept_teenagers,...,young_response_count,young_female_response_count,young_male_response_count,invitation_count,female_invitation_count,male_invitation_count,young_invitation_count,young_female_invitation_count,young_male_invitation_count,industry_id_list
49839,81902096,2818876,0,2023-06-14,shift,volonteer,up_to_3_years,35000.0,50000.0,False,...,0,0,0,12,4,8,0,0,0,[None]
145838,80948808,3036416,100,2023-05-24,flexible,volonteer,no_experience,NaN,69000.0,False,...,67,50,17,1,1,0,0,0,0,[44]
227995,77374703,8980856,1,2023-02-24,flexible,volonteer,no_experience,NaN,NaN,False,...,0,0,0,4,0,4,0,0,0,[None]
357412,77073973,2818876,0,2023-02-16,shift,volonteer,up_to_3_years,35000.0,50000.0,False,...,0,0,0,10,0,10,0,0,0,[None]


### Также запишем новый категориальный столбец, который содержит информацию о том, содержались ли nan в зарплате

In [29]:
data[['compensation_from', 'compensation_to']] = data[['compensation_from', 'compensation_to']].replace(0, np.nan)

In [30]:
mask = data['compensation_from'].isna() | data['compensation_to'].isna()

In [31]:
data['correct_salary'] = mask.apply(lambda x: 'approximately' if x else 'correct')

In [32]:
data

,vacancy_id,employer_id,employees_number,creation_date,work_schedule,employment,length_of_employment,compensation_from,compensation_to,accept_teenagers,...,young_female_response_count,young_male_response_count,invitation_count,female_invitation_count,male_invitation_count,young_invitation_count,young_female_invitation_count,young_male_invitation_count,industry_id_list,correct_salary
0,77424192,2571258,300,2023-02-27,full_day,full,up_to_3_years,35000.0,NaN,False,...,4,0,21,21,0,0,0,0,[48],approximately
1,78161908,4809555,50,2023-03-17,shift,full,no_experience,22500.0,NaN,False,...,4,1,26,26,0,2,2,0,[None],approximately
2,79496395,5667513,5,2023-04-19,shift,full,up_to_3_years,24000.0,30000.0,False,...,2,0,3,3,0,0,0,0,[None],correct
3,87102246,1072323,0,2023-09-21,full_day,full,up_to_3_years,50000.0,NaN,False,...,0,0,9,7,2,0,0,0,"[7, 8]",approximately
4,76367381,1160192,0,2023-01-30,full_day,full,up_to_3_years,55000.0,150000.0,False,...,0,0,49,39,10,0,0,0,"[5, 13]",correct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377760,87825374,3558,7000,2023-10-06,flyInFlyOut,full,up_to_3_years,96780.0,NaN,False,...,0,0,9,8,1,0,0,0,[50],approximately
377761,77397135,4446306,2,2023-02-27,remote,full,above_6_years,100000.0,150000.0,False,...,0,0,5,3,2,0,0,0,[7],correct
377762,75924698,2067104,0,2023-01-18,shift,full,up_to_3_years,35000.0,40000.0,False,...,0,0,34,34,0,0,0,0,[None],correct
377763,84486705,1838,0,2023-08-03,full_day,full,up_to_3_years,85000.0,NaN,False,...,0,0,1,0,1,0,0,0,[44],approximately


Теперь заменим все nans в compensation_from на медианную зарплату данной профессии. Такое решение содержит множество допущений, ведь есть множество факторов, влияющих на 'compensation_from', но 'specialization' мы посчитали наиболее значимым и достаточным (как аргумент, там нет пропусков и на каждое уникальное значение в среднем 15000 сэмплов)

Технически также можно взять медианную более мелких групп, вроде сочетаний профессии и региона, профессии и работодателя, но эти группы могут быть крайне маленькими или вообще не быть, что неинформативно, поэтому мы посчитали, что только профессии будет достаточно.

##### Также мы посчитали коэффициент корреляции eta для пар: 'compensation_from' и 'specialization', 'compensation_from' и 'region_name'

In [33]:
def get_eta_correlation(categorical_col):
    groups = [data['compensation_from'][data[categorical_col] == value] for value in data[categorical_col].unique()]
        
    total_variance = np.var(data['compensation_from'])
    group_means = [g.mean() for g in groups]
    group_sizes = [len(g) for g in groups]
    group_variance = sum(size * (mean - data['compensation_from'].mean()) ** 2 for size, mean in zip(group_sizes, group_means)) / len(data)

    return np.round(group_variance / total_variance, 3)

In [34]:
get_eta_correlation('specialization')

0.274

In [35]:
get_eta_correlation('region_name')

0.024

Заметим, что специализация влияет на зарплату гораздо значимее, чем регион

In [36]:
data['compensation_from'] = data['compensation_from'].fillna(
    data.groupby('specialization')['compensation_from'].transform('median')
)

In [37]:
'''случаи, где compensation_from > compensation_to'''

data[(data['compensation_from'] > data['compensation_to']) & (data['compensation_to'] != 0)].shape[0]

4438

Также есть данные, где compensation_from, в которой мы заполняли пропуски, больше чем compensation_to. В таких случаях мы опустим compensation_from до уровня compensation_to

In [38]:
data.loc[(data['compensation_from'] > data['compensation_to']) & (data['compensation_to'] != 0), 'compensation_from'] = data['compensation_to']

In [39]:
data['compensation_to'] = data['compensation_to'].fillna(data['compensation_from'])

In [40]:
sum(data.isnull().sum())

0

# Дополнительные колонки

Добавим данные о длине списка id индустрий

In [41]:
data['industry_id_list'] = data['industry_id_list'].apply(eval)

In [42]:
data['length_industry_list'] = data['industry_id_list'].apply(lambda x: len(x) if int(bool(x[0])) else 0)

In [43]:
data['length_industry_list'].value_counts()

length_industry_list
0    151223
1    140631
2     61242
3     12529
4      2275
5       379
Name: count, dtype: int64

# Сохранение датасета

In [44]:
data.to_csv('cleared_data_no_exp.csv', index=False)

In [45]:
pd.read_csv('cleared_data_no_exp.csv').describe()

,vacancy_id,employer_id,employees_number,compensation_from,compensation_to,response_count,female_response_count,male_response_count,young_response_count,young_female_response_count,young_male_response_count,invitation_count,female_invitation_count,male_invitation_count,young_invitation_count,young_female_invitation_count,young_male_invitation_count,length_industry_list
count,3.682790e+05,3.682790e+05,368279.000000,3.682790e+05,3.682790e+05,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000,368279.000000
mean,8.355033e+07,3.527593e+06,9426.058950,5.372587e+04,6.948788e+04,22.787552,13.474485,9.313067,1.811485,1.177488,0.633998,17.453067,10.070175,7.382892,0.911184,0.583080,0.328105,0.846361
std,4.569188e+06,3.134877e+06,45434.024982,3.177095e+04,5.049795e+04,52.050955,39.276758,20.838503,7.742204,5.330763,2.933670,48.294500,35.407356,20.737687,5.741346,3.834673,2.205923,0.872613
min,7.453818e+07,6.000000e+00,0.000000,1.000000e+00,1.300000e+01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.964190e+07,8.597140e+05,1.000000,3.500000e+04,4.000000e+04,4.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.360487e+07,2.967686e+06,50.000000,4.500000e+04,5.000000e+04,10.000000,4.000000,4.000000,0.000000,0.000000,0.000000,6.000000,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000
75%,8.767547e+07,5.195285e+06,1100.000000,6.000000e+04,8.000000e+04,22.000000,12.000000,9.000000,1.000000,0.000000,0.000000,17.000000,9.000000,6.000000,0.000000,0.000000,0.000000,1.000000
max,9.116080e+07,1.123410e+07,682000.000000,1.500000e+06,3.000000e+06,5538.000000,4534.000000,1014.000000,608.000000,399.000000,231.000000,5805.000000,4752.000000,2007.000000,691.000000,434.000000,257.000000,5.000000
